## Felix Gaber   // 20221385
## David Halder //  20220632

# Message to the Professors: 
Please review our submission of question 6: 
"Considering only the last reviewed order of each customer, what is the percentage of reviewed orders that have a review_score higher than 4 for each city? **(order by percentage, ascending)**"

Our submission was the following:  
RIVENDELL 72.7 ROHAN 73.5 THE SHIRE 76.2 GONDOR 76.2 MORDOR 76.4

In the review of our submission, it says that the correct answer is: 
The correct answer is: THE SHIRE 76.2 RIVENDELL 72.7 ROHAN 73.5 MORDOR 76.4 GONDOR 76.2

We gave the same values in our submission, but we think that our answer is right, because the question clearly states: **order by percentage, ascending**. 
We kindly request that you review our response. If you believe that our answer is incorrect, please do not hesitate to provide us with an explanation as to why it may be inaccurate. Your feedback is greatly appreciated.

Step 1: Download data from dropbox into local storage.

In [0]:
%sh
wget bda_data.zip https://www.dropbox.com/s/uuwrpkpdktajpnk/bda_data.zip

--2023-03-19 12:58:23--  http://bda_data.zip/
Resolving bda_data.zip (bda_data.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘bda_data.zip’
--2023-03-19 12:58:23--  https://www.dropbox.com/s/uuwrpkpdktajpnk/bda_data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/uuwrpkpdktajpnk/bda_data.zip [following]
--2023-03-19 12:58:23--  https://www.dropbox.com/s/raw/uuwrpkpdktajpnk/bda_data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0ebab4cdf1453e6704093f9a9a.dl.dropboxusercontent.com/cd/0/inline/B4hbVg6A7nfuomSCvzXmwCxSV8I14ATchU3oYYP5r-4yitFxaokVTG2AzLUIco7rju2CDBW8LkKMQKuk7w-fegTcDCTn9ObdD8DSh-1F5DuH4cWssPcNFz-XT57epR1gIhXyymgMNJyNovmmS9zEQEzT0neBWF9opCAkqWCSvFuu7A/file# [following]
--2023

In [0]:
%sh
ls

azure
bda_data.zip
conf
eventlogs
ganglia
hadoop_accessed_config.lst
logs
metastore_db
preload_class.lst


Step 2: Unzip the file

In [0]:
%sh
unzip -o bda_data.zip

Archive:  bda_data.zip
   creating: bda_data/
   creating: bda_data/mkt_test_assignment_sample/
  inflating: bda_data/mkt_test_assignment_sample/part-00030-tid-8709977867890689006-0f28043d-8406-4057-ae46-2eb1a0283329-4573-1-c000.snappy.parquet  
  inflating: bda_data/mkt_test_assignment_sample/part-00026-tid-8709977867890689006-0f28043d-8406-4057-ae46-2eb1a0283329-4569-1-c000.snappy.parquet  
 extracting: bda_data/mkt_test_assignment_sample/_SUCCESS  
  inflating: bda_data/mkt_test_assignment_sample/part-00023-tid-8709977867890689006-0f28043d-8406-4057-ae46-2eb1a0283329-4592-1-c000.snappy.parquet  
  inflating: bda_data/mkt_test_assignment_sample/part-00017-tid-8709977867890689006-0f28043d-8406-4057-ae46-2eb1a0283329-4595-1-c000.snappy.parquet  
  inflating: bda_data/mkt_test_assignment_sample/part-00010-tid-8709977867890689006-0f28043d-8406-4057-ae46-2eb1a0283329-4586-1-c000.snappy.parquet  
  inflating: bda_data/mkt_test_assignment_sample/part-00000-tid-8709977867890689006-0f28043d-8

Step 3: Copy file into DBFS

In [0]:
%fs
cp -r file:/databricks/driver/bda_data/ FileStore/bda_data

res5: Boolean = true

In [0]:
%fs
ls FileStore

path,name,size,modificationTime
dbfs:/FileStore/LOTR_1.txt,LOTR_1.txt,1025382,1678215327000
dbfs:/FileStore/bda_data/,bda_data/,0,0
dbfs:/FileStore/bda_data.parquet,bda_data.parquet,1381815006,1678982251000
dbfs:/FileStore/bda_data.zip,bda_data.zip,402111594,1678980667000
dbfs:/FileStore/dave/,dave/,0,0
dbfs:/FileStore/tables/,tables/,0,0
dbfs:/FileStore/tb.csv,tb.csv,1003544,1678316661000


In [0]:
%fs
ls /FileStore/bda_data/

path,name,size,modificationTime
dbfs:/FileStore/bda_data/items_sample/,items_sample/,0,0
dbfs:/FileStore/bda_data/mkt_test_assignment_sample/,mkt_test_assignment_sample/,0,0
dbfs:/FileStore/bda_data/ord_sample/,ord_sample/,0,0
dbfs:/FileStore/bda_data/sess_sample/,sess_sample/,0,0


Step 4: Create temporary views for each of the respective tables

In [0]:
# Information about ordered items: 
items_sample = sqlContext.read.parquet("dbfs:/FileStore/bda_data/items_sample/*.snappy.parquet")
items_sample.createOrReplaceTempView("items_sample")

# Information about testing sessions:
mkt_test_assignment_sample = sqlContext.read.parquet("dbfs:/FileStore/bda_data/mkt_test_assignment_sample/*.snappy.parquet")
mkt_test_assignment_sample.createOrReplaceTempView("mkt_test_assignment_sample")

# Information about orders:
ord_sample = sqlContext.read.parquet("dbfs:/FileStore/bda_data/ord_sample/*.snappy.parquet")
ord_sample.createOrReplaceTempView("ord_sample")

# Information about sessions:
sess_sample = sqlContext.read.parquet("dbfs:/FileStore/bda_data/sess_sample/*.snappy.parquet")
sess_sample.createOrReplaceTempView("sess_sample")

Step 5: Answering the questions

Question 1: What proportion of users who view a restaurant also view a dish, broken by platform?

In [0]:
%sql
WITH restaurant AS (
  SELECT platform, 
  COUNT(account_id) AS rest
  FROM sess_sample
  WHERE view_restaurant = TRUE 
  GROUP BY 1
), 

rest_dish AS (
  SELECT 
  platform, 
  COUNT(account_id) AS rest_dish
  FROM sess_sample
  WHERE view_dish = TRUE 
  AND view_restaurant = TRUE 
  GROUP BY 1
) 

SELECT 
  platform, 
  ROUND(rest_dish/rest, 1) AS prop
FROM restaurant 
JOIN rest_dish USING (platform)


platform,prop
ANDROID,0.8
IOS,0.8
WEB,0.8


Question 2: Who is the account ID with highest amount of RESTAURANT orders?

In [0]:
%sql
SELECT 
  account_id, 
  COUNT(*) AS restaurant_orders
FROM ord_sample
WHERE order_origin = 'RESTAURANT'
GROUP BY account_id
ORDER BY restaurant_orders DESC
LIMIT 1

account_id,restaurant_orders
59a4daa0619c067d79f9c0be27dfe61d91bad7058dea5cd5e964482409484f50,429


Question 3: What is the difference in the average review_score from the first month to the last month for each order_origin? (order by difference, ascending)

In this question, we have two possible solutions: 
1. Using the column reference_date to extract first and last month
2. using the column review_created_at to axtract first and last month

--> As we want to connect the review to the date when the order was fulfilled, we decided to proceed with the first option.

In [0]:
%sql
-- 1. Using the column reference_date to extract first and last month
WITH first_month AS (
SELECT 
  order_origin, 
  AVG(review_score) AS first_month_avg_review
FROM ord_sample
WHERE MONTH(reference_date) = (SELECT MIN(MONTH(reference_date)) FROM ord_sample)
GROUP BY order_origin
),

last_month AS (
SELECT 
  order_origin, 
  AVG(review_score) AS last_month_avg_review
FROM ord_sample
WHERE MONTH(reference_date) = (SELECT MAX(MONTH(reference_date)) FROM ord_sample)
GROUP BY order_origin
)

SELECT 
  order_origin, 
  round((last_month.last_month_avg_review - first_month.first_month_avg_review), 1) AS difference
FROM first_month
JOIN last_month USING (order_origin)
ORDER BY difference ASC

order_origin,difference
OTHERS,-0.1
STORE,0.1
RESTAURANT,0.1


In [0]:
%sql
-- 2. using the column review_created_at to axtract first and last month
WITH fm AS (
SELECT
  order_origin,
  AVG(review_score) AS score_fm
FROM ord_sample
WHERE DATE_TRUNC("month", review_created_at) = (SELECT MIN(DATE_TRUNC("month", review_created_at)) FROM ord_sample)
AND review_score IS NOT NULL
GROUP BY 1
),

lm AS (
SELECT
  order_origin,
  AVG(review_score) AS score_lm
FROM ord_sample
WHERE DATE_TRUNC("month", review_created_at) = (SELECT MAX(DATE_TRUNC("month", review_created_at)) FROM ord_sample)
AND review_score IS NOT NULL
GROUP BY 1
)

SELECT
order_origin,
score_fm,
score_lm,
ROUND(score_lm - score_fm, 1) diff
FROM lm
JOIN fm USING(order_origin)
ORDER BY diff ASC


order_origin,score_fm,score_lm,diff
RESTAURANT,4.5737908365197555,4.764457577036724,0.2
STORE,4.475396163469558,4.90566037735849,0.4
OTHERS,4.192982456140351,5.0,0.8


Question 4: What is the cumulative sum of order_total for each month over time? (order by date, ascending)

In [0]:
%sql
WITH monthly_order_total AS (
  SELECT 
    date_trunc("month", reference_date) AS month, 
    SUM(order_total) AS total
  FROM ord_sample
  GROUP BY month
),

cumulative_order_total AS (
  SELECT 
    month, 
    total,
    SUM(total) OVER (ORDER BY month ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_total
  FROM
    monthly_order_total
)

SELECT
  date(month) as month,
  round(cumulative_total, 1) as cumulative_total
FROM
  cumulative_order_total
ORDER BY
  1 ASC;

month,cumulative_total
2021-01-01,6.667353767E8
2021-02-01,1.2755102965E9
2021-03-01,2.0581694616E9
2021-04-01,2.8510102824E9
2021-05-01,3.6319924641E9
2021-06-01,4.3583445969E9


Question 5: How many orders from STORES on June/21, in MORDOR, received any reviews?

In [0]:
%sql

SELECT 
  COUNT(order_id) AS num_reviews
FROM ord_sample
WHERE MONTH(reference_date) = 6
  AND YEAR(reference_date) = 2021
  AND city = 'MORDOR'
  AND order_origin = 'STORE'
  AND review_score IS NOT NULL

num_reviews
296


Question 6: Considering only the last reviewed order of each customer, what is the percentage of orders that have a review_score higher than 4 for each city? (order by percentage, ascending)

First we are filtering the dataset to only be left with orders who got any kind of review. Next, we are using the filtered dataset to get the score, date and city of the last review per account. These values are then filtered to be left only with values which have a review score over four. Next, these tables are joined together and the percentage of orders who have a higher review_score than 4 for each city is calculated. Finally the cities are ordered by the retrieved percentages in an ascending order.

In [0]:
%sql

WITH reviewed_orders AS (
  SELECT 
    city, 
    review_score, 
    account_id, 
    review_created_at
  FROM ord_sample
  WHERE review_score IS NOT NULL
),

last_reviewed_orders AS (
  SELECT city, review_score, account_id,
         ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY review_created_at DESC) AS rn
  FROM reviewed_orders
),

last_orders AS (
  SELECT 
    city, 
    review_score
  FROM last_reviewed_orders
  WHERE rn = 1
),

positive_reviews AS (
  SELECT 
    city, 
    COUNT(*) AS num_positive_reviews
  FROM last_orders
  WHERE review_score > 4
  GROUP BY city
),

total_last_orders AS (
  SELECT 
    city, 
    COUNT(*) AS num_last_orders
  FROM last_orders
  GROUP BY city
)

SELECT p.city, 
       round(100.0 * p.num_positive_reviews / t.num_last_orders, 1) AS percentage_positive_reviews
FROM positive_reviews p
JOIN total_last_orders t ON p.city = t.city
ORDER BY percentage_positive_reviews ASC;


city,percentage_positive_reviews
RIVENDELL,72.7
ROHAN,73.5
THE SHIRE,76.2
GONDOR,76.2
MORDOR,76.4


Question 7: What is the percentage change of the total number of orders from the first month to the last month?

In [0]:
%sql
WITH first_month AS (
  SELECT 
    DATE_TRUNC('month', MIN(reference_date)) AS month_start
  FROM ord_sample
),

last_month AS (
  SELECT 
    DATE_TRUNC('month', MAX(reference_date)) AS month_start
  FROM ord_sample
),

first_month_orders AS (
  SELECT 
    COUNT(order_id) AS total_orders
  FROM ord_sample
  WHERE DATE_TRUNC('month', reference_date) = (SELECT month_start FROM first_month)
),

last_month_orders AS (
  SELECT 
    COUNT(order_id) AS total_orders
  FROM ord_sample
  WHERE DATE_TRUNC('month', reference_date) = (SELECT month_start FROM last_month)
)

SELECT 
  round((l.total_orders - f.total_orders) * 100.0 / f.total_orders,1) AS percentage_change
FROM first_month_orders f, last_month_orders l;


percentage_change
6.4


Question 8: What is the average order value of the first order of each customer in RIVENDELL?

In this question, we have two different possibilities to solve the question: 
1. We filter first for the first order, and afterwards for the order made in Rivendell
2. We filter first for the order made in Rivendell, and afterwards for the first order
--> We decided to hand in solution 1

In [0]:
%sql 
-- Solution 1: 
WITH first_orders AS (
SELECT
  account_id,
  MIN(reference_date) AS first_order_date
FROM ord_sample
GROUP BY 1
)


SELECT 
  City,  
  ROUND(AVG(order_total), 1) AS order_total
FROM ord_sample
LEFT JOIN first_orders USING (account_id)
WHERE reference_date = first_order_date
AND city = "RIVENDELL"
GROUP BY 1

City,order_total
RIVENDELL,2422.2


In [0]:
%sql
-- Solution 2: We filter first for the order made in Rivendell, and afterwards for the first order
WITH RIV_HELP AS(
SELECT *
FROM ORD_SAMPLE
WHERE CITY = "RIVENDELL"
),

FIRST_ORDER_TABLE AS (
  SELECT
    ACCOUNT_ID, 
    ORDER_TOTAL, 
    ROW_NUMBER() OVER (PARTITION BY ACCOUNT_ID ORDER BY REFERENCE_DATE ASC) AS ORD
  FROM RIV_HELP
)

SELECT 
  ROUND(AVG(ORDER_TOTAL),1) FIRST_ORDER_AVERAGE
FROM FIRST_ORDER_TABLE
WHERE ORD = 1

FIRST_ORDER_AVERAGE
2456.8


Question 9: Which order_shift had the highest average delivery fee?

In [0]:
%sql

SELECT 
  order_shift, 
  round(AVG(delivery_fee),1) AS avg_delivery_fee
FROM ord_sample
GROUP BY order_shift
ORDER BY avg_delivery_fee DESC
LIMIT 1

order_shift,avg_delivery_fee
weekend lunch,188.7


Question 10: What is the average distance between the merchant and customer for each order origin?

In [0]:
%sql 
SELECT 
  order_origin, 
  ROUND(AVG(distance_merchant_customer), 1) AS avg_dist
FROM ord_sample
GROUP BY order_origin
ORDER BY avg_dist ASC


order_origin,avg_dist
OTHERS,3186.1
RESTAURANT,4806.3
STORE,5263.6
